<a href="https://colab.research.google.com/github/artem-dorofeev/py_DataScience_HW/blob/main/HW12_whith_ukr_add.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Домашнє завдання 12**

*Зробіть summary нижчевказаного тексту використовуючи бібліотеки для NLP: nltk та SpaCy*

In [1]:
# імпортування бібліотек

import spacy

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize

import string
from heapq import nlargest

In [2]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

*Спробую два методи нижчі опишу різницю.*

In [3]:
# Перш ніж почати роботу з SpaCy, необхідно завантажити потрібну мовну модель.

nlp = spacy.load('en_core_web_sm')

In [4]:
# Текст для обробки
text = "The Orbiter Discovery, OV-103, is considered eligible for listing in the National Register of Historic Places (NRHP) in the context of the U.S. Space Shuttle Program (1969-2011) under Criterion A in the areas of Space Exploration and Transportation and under Criterion C in the area of Engineering. Because it has achieved significance within the past fifty years, Criteria Consideration G applies. Under Criterion A, Discovery is significant as the oldest of the three extant orbiter vehicles constructed for the Space Shuttle Program (SSP), the longest running American space program to date; she was the third of five orbiters built by NASA. Unlike the Mercury, Gemini, and Apollo programs, the SSP’s emphasis was on cost effectiveness and reusability, and eventually the construction of a space station. Including her maiden voyage (launched August 30, 1984), Discovery flew to space thirty-nine times, more than any of the other four orbiters; she was also the first orbiter to fly twenty missions. She had the honor of being chosen as the Return to Flight vehicle after both the Challenger and Columbia accidents. Discovery was the first shuttle to fly with the redesigned SRBs, a result of the Challenger accident, and the first shuttle to fly with the Phase II and Block I SSME. Discovery also carried the Hubble Space Telescope to orbit and performed two of the five servicing missions to the observatory. She flew the first and last dedicated Department of Defense (DoD) missions, as well as the first unclassified defense-related mission. In addition, Discovery was vital to the construction of the International Space Station (ISS); she flew thirteen of the thirty-seven total missions flown to the station by a U.S. Space Shuttle. She was the first orbiter to dock to the ISS, and the first to perform an exchange of a resident crew. Under Criterion C, Discovery is significant as a feat of engineering. According to Wayne Hale, a flight director from Johnson Space Center, the Space Shuttle orbiter represents a “huge technological leap from expendable rockets and capsules to a reusable, winged, hypersonic, cargo-carrying spacecraft.” Although her base structure followed a conventional aircraft design, she used advanced materials that both minimized her weight for cargo-carrying purposes and featured low thermal expansion ratios, which provided a stable base for her Thermal Protection System (TPS) materials. The Space Shuttle orbiter also featured the first reusable TPS; all previous spaceflight vehicles had a single-use, ablative heat shield. Other notable engineering achievements of the orbiter included the first reusable orbital propulsion system, and the first two-fault-tolerant Integrated Avionics System. As Hale stated, the Space Shuttle remains “the largest, fastest, winged hypersonic aircraft in history,” having regularly flown at twenty-five times the speed of sound."

In [6]:
# Токенізація
doc = nlp(text)
# tokens = [token.text for token in doc]
# print(tokens)


In [7]:
# За допомогою методів NLTK, таких як word_tokenize, sent_tokenize або stopwords, ми можемо отримати токенізовані слова та речення, а також список стоп-слів

tokens = word_tokenize(text)
sentences = sent_tokenize(text)
stop_words = set(stopwords.words('english'))

# І також не забуваємо про знаки пунктуації
punctuation = string.punctuation + '\n'

In [ ]:
tokens

In [9]:
punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~\n'

In [10]:
# Також можна порахувати частоту появи тих чи інших слів у тексті(але варто пам'ятати, що це треба робити, вже виключивши усі знаки пунктуації)

word_frequencies = {}
for word in doc:
    if word.text.lower() not in stop_words and word.text.lower() not in punctuation:
        if word.text not in word_frequencies.keys():
            word_frequencies[word.text] = 1
        else:
            word_frequencies[word.text] += 1

In [ ]:
# сортуємо та виводимо на екран
sorted_word_frequencies = sorted(word_frequencies.items(), key=lambda x: x[1], reverse=True)
sorted_word_frequencies

In [ ]:
word_frequencies

*У першому методі ви обчислюєте "вагову оцінку" кожного речення, враховуючи кількість зустрічаних у ньому слів, які зустрічаються у словнику частот слів:*

In [13]:
sentence_scores = {}
for sent in sentences:
    for word in nltk.word_tokenize(sent.lower()):
        if word in word_frequencies.keys():
            if len(sent.split(' ')) < 30:  # Filter out very long sentences
                if sent not in sentence_scores.keys():
                    sentence_scores[sent] = word_frequencies[word]
                else:
                    sentence_scores[sent] += word_frequencies[word]

In [14]:
summary_sentences = nlargest(3, sentence_scores, key=sentence_scores.get)
summary_sentences

['Other notable engineering achievements of the orbiter included the first reusable orbital propulsion system, and the first two-fault-tolerant Integrated Avionics System.',
 'The Space Shuttle orbiter also featured the first reusable TPS; all previous spaceflight vehicles had a single-use, ablative heat shield.',
 'She flew the first and last dedicated Department of Defense (DoD) missions, as well as the first unclassified defense-related mission.']

*У другому методі ви використовуєте функцію схожості SpaCy для порівняння кожного речення з усім текстом. Це оцінює схожість між кожним реченням та усім текстом, а не просто рахує кількість зустрічаних у реченні слів:*

In [15]:
# Вагова оцінка речень з використанням SpaCy
def rank_sentences_spacy(text, num_sentences=3):
    doc = nlp(text)
    sentences = [sent.text for sent in doc.sents] # Кожне речення у тексті витягується окремо за допомогою doc.sents, та перетворюється на рядок
    ranked_sentences = sorted(sentences, key=lambda x: nlp(x).similarity(doc), reverse=True) # Речення сортуються за їх ваговою оцінкою, яка визначається з використанням nlp(x).similarity(doc), де x - це кожне окреме речення у тексті
    return ranked_sentences[:num_sentences]

# Вибір найважливіших речень з використанням SpaCy
important_sentences_spacy = rank_sentences_spacy(text)


<ipython-input-15-049bea8bb49b>:5: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  ranked_sentences = sorted(sentences, key=lambda x: nlp(x).similarity(doc), reverse=True) # Речення сортуються за їх ваговою оцінкою, яка визначається з використанням nlp(x).similarity(doc), де x - це кожне окреме речення у тексті


In [16]:
# Створення резюме на основі найважливіших речень
summary_spacy = ' '.join(important_sentences_spacy)

summary_spacy

'Under Criterion A, Discovery is significant as the oldest of the three extant orbiter vehicles constructed for the Space Shuttle Program (SSP), the longest running American space program to date; she was the third of five orbiters built by NASA. In addition, Discovery was vital to the construction of the International Space Station (ISS); she flew thirteen of the thirty-seven total missions flown to the station by a U.S. Space Shuttle. According to Wayne Hale, a flight director from Johnson Space Center, the Space Shuttle orbiter represents a “huge technological leap from expendable rockets and capsules to a reusable, winged, hypersonic, cargo-carrying spacecraft.”'

*Результати будуть різними, оскільки ці два підходи вимірюють різні аспекти "важливості" речень. У першому випадку "важливість" вимірюється частотою слів, які зустрічаються в реченні. У другому випадку "важливість" вимірюється ступенем схожості речення з усім текстом.*

*спробуємо для тексту українською мовою*

In [17]:
# Завантаження файлу
file_path = '/content/data/dges.txt'

# Читання вмісту файлу
with open(file_path, 'r', encoding='utf-8') as file:
    text_ua = file.read()

# Обробка тексту за допомогою spaCy
# doc_ua = nlp(text_ua)


In [ ]:
doc_ua

In [18]:
text_ua

"Дніпровська гідроелектростанція – найстаріша ГЕС, побудована на річці Дніпро. Її з впевненістю можна вважати символом цілої епохи, який і зараз викликає захоплення своєю красою та величчю. Досі дивує, як таку масштабну споруду вдалося збудувати ще у 20-х роках ХХ століття. Історія будівництва Дніпровської ГЕС дуже цікава. З цим проєктом пов’язано безліч міфів та легенд.\n\nІдея будівництва греблі на річці Дніпро виникла ще в ХVІІІ столітті, у часи правління Катерини ІІ. Після захоплення Криму та Таврії імперія розглядала Дніпро як перспективну транспорту артерію для постачання товарів на нові землі та укріплення своїх позицій на сході. Але такому використанню річки заважали відомі дніпровські пороги, тому пріоритетом у проєктах того часу було забезпечення судноплавства по всій довжині Дніпра, шляхом їх затоплення. У 1778 році почалася історія боротьби з порогами технічними засобами – були здійснені спроби підірвати пороховими зарядами Старо-Кайдацький поріг та Ненаситець. За півтора с

In [19]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer

# Завантаження токенізатора для української мови
nltk.download('perluniprops')
nltk.download('nonbreaking_prefixes')

[nltk_data] Downloading package perluniprops to /root/nltk_data...
[nltk_data]   Unzipping misc/perluniprops.zip.
[nltk_data] Downloading package nonbreaking_prefixes to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping corpora/nonbreaking_prefixes.zip.


True

In [ ]:
# Використання токенізатора для української мови
# text_ua = "Привіт, як справи?"
tokenizer = RegexpTokenizer(r'\w+')
tokens_ua = tokenizer.tokenize(text_ua)
tokens_ua

In [22]:
#  бібліотека stop-words, яка містить словники стоп-слів для багатьох мов, включаючи українську

!pip install stop-words

  Preparing metadata (setup.py) ... done
  Created wheel for stop-words: filename=stop_words-2018.7.23-py3-none-any.whl size=32895 sha256=4ab881cd416c66f1cf574d36aa214e792c91b532411de242795acd77614eac4b
  Stored in directory: /root/.cache/pip/wheels/d0/1a/23/f12552a50cb09bcc1694a5ebb6c2cd5f2a0311de2b8c3d9a89
Successfully built stop-words


In [23]:
from stop_words import get_stop_words

# Отримання множини стоп-слів для української мови
stop_words_ua = set(get_stop_words('ukrainian'))


 *NLTK не має вбудованих моделей для української мови для модуля punkt. Однак, можливо скористатися альтернативними бібліотеками, які мають підтримку української мови, такими як razdel або tokenizers. *

In [24]:
!pip install razdel


In [25]:
from razdel import sentenize

sentences_ua = [sentence.text for sentence in sentenize(text_ua)]
print(sentences_ua)


['Дніпровська гідроелектростанція – найстаріша ГЕС, побудована на річці Дніпро.', 'Її з впевненістю можна вважати символом цілої епохи, який і зараз викликає захоплення своєю красою та величчю.', 'Досі дивує, як таку масштабну споруду вдалося збудувати ще у 20-х роках ХХ століття.', 'Історія будівництва Дніпровської ГЕС дуже цікава.', 'З цим проєктом пов’язано безліч міфів та легенд.', 'Ідея будівництва греблі на річці Дніпро виникла ще в ХVІІІ столітті, у часи правління Катерини ІІ.', 'Після захоплення Криму та Таврії імперія розглядала Дніпро як перспективну транспорту артерію для постачання товарів на нові землі та укріплення своїх позицій на сході.', 'Але такому використанню річки заважали відомі дніпровські пороги, тому пріоритетом у проєктах того часу було забезпечення судноплавства по всій довжині Дніпра, шляхом їх затоплення.', 'У 1778 році почалася історія боротьби з порогами технічними засобами – були здійснені спроби підірвати пороховими зарядами Старо-Кайдацький поріг та Не

In [26]:
# встановлення бібліотекі summa

!pip install summa

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.9/54.9 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for summa: filename=summa-1.2.0-py3-none-any.whl size=54386 sha256=006bbcc55f4597b913e33147e09f6f8549e1942dde3e46e58d03d80af834cdba
  Stored in directory: /root/.cache/pip/wheels/4a/ca/c5/4958614cfba88ed6ceb7cb5a849f9f89f9ac49971616bc919f
Successfully built summa


In [27]:
from summa.summarizer import summarize

# Обробка тексту та отримання summary
summary_ua = summarize(text_ua, ratio=0.1)  # Вибрати 10% найбільш важливих речень як summary

print(summary_ua)


Ідея будівництва греблі на річці Дніпро виникла ще в ХVІІІ столітті, у часи правління Катерини ІІ.
У 1778 році почалася історія боротьби з порогами технічними засобами – були здійснені спроби підірвати пороховими зарядами Старо-Кайдацький поріг та Ненаситець.
На довгі роки країна поринула у війну, революції та руїну.
Зміни на краще почалися тут в епоху індустріалізації, коли Олександрівськ став однією з найперших та найважливіших ланок перспективного плану електрифікації країни.
Саме голові відомої на той час у всьому світі американської організації з проєктування та будівництва гідроелектростанцій, Х’ю Куперу (H.L.Cooper & Co), ми завдячуємо тим, що гребля має форму підкови.
Розуміючи значення для будівництва кваліфікованих кадрів, керівництво Дніпробуду на об’єкті почало створювати загальноосвітні школи та курси з підготовки спеціалістів - машиністів, слюсарів, теслярів та інших фахівців.
Десятки та сотні робітників, які свердлять камінь берегів Дніпра електричними свердлами, підрива